In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
vm = input()
if vm.lower()=='yes':
    vm=True
else:
    vm=False

yes


In [3]:
train_mode = input()
if train_mode.lower()=='yes':
    train_mode=True
else:
    train_mode=False

no


In [4]:
if vm:
    path='/mnt/cephfs/ml_data/mc_2021/'
else:
    path=''
    
data_real = pd.read_csv('{}processed_data/ProcessedTrainReal/ProcessedTrain_1M.csv.gz'.format(path))
data_real = data_real[data_real['edepR'] < 17.2]

In [5]:
from sklearn.metrics import mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
size = int(1e6)
n_feats = len(data_real.columns) - 5

X_val = data_real.iloc[:, :-5][size:]
y_val = data_real.iloc[:, -5][size:]
data_real = data_real[:size]

In [7]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

if train_mode:    
    n_folds = 5
    rmse_scores = []
    mape_scores = []
    
    kfold = KFold(n_folds, True, random_state=22)    
    for train, test in tqdm(kfold.split(data_real), "Folds... ", leave=False):        
        xgbreg = XGBRegressor(
                max_depth=9,
                learning_rate=0.08,
                n_estimators=3000,
        )
                            
        X_train = np.array(data_real)[train][:, :n_feats]
        y_train = np.array(data_real)[train][:, n_feats]
                            
        X_test = np.array(data_real)[test][:, :n_feats]
        y_test = np.array(data_real)[test][:, n_feats]

        xgbreg.fit(X_train, y_train,
                   verbose=False,
                   eval_set=[(np.array(X_val), np.array(y_val))],
                   early_stopping_rounds=3)
        
        y_predict = xgbreg.predict(X_test)
        rmse = mean_squared_error(y_predict, y_test)**0.5
        mape = mean_absolute_percentage_error(y_predict, y_test)
        rmse_scores.append(rmse)
        mape_scores.append(mape)
    
    result = np.array([[np.mean(mape_scores), np.std(mape_scores)], [np.mean(rmse_scores), np.std(rmse_scores)]])
    np.savez_compressed('feature_selection/all_features_metrics.npz', a=result)

In [8]:
all_features_metrics = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a']
all_features_metrics

array([[1.17233915e+00, 9.15770732e-04],
       [8.25936170e-02, 2.74837512e-04]])

In [9]:
all_features_metric = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0][0]
eps = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0][1]

opt_features = []
current_metrics = []
current_metric_stds = []
current_metric = 100

features = data_real.iloc[:, :-5].columns
features = features.drop(opt_features)
while abs(all_features_metric - current_metric) > eps:
    metrics = []
    metric_stds = []
    for feature in tqdm(features[len(metrics):], "Features loop"):
        
        X = data_real.iloc[:, :-5][opt_features+[feature]]
        y = data_real.iloc[:, -5]
        
        xgbreg = XGBRegressor(
            max_depth=9,
            learning_rate=0.08,
            n_estimators=3000,
            random_state=22,
        )
        
        scores = cross_val_score(
            xgbreg,
            X,
            y,
            cv=5,
            n_jobs=-1,
            verbose=False,
            fit_params={
                'eval_set': [(X_val[opt_features+[feature]], y_val)],
                'early_stopping_rounds':3
            },
            scoring='neg_mean_absolute_percentage_error'
        )
        
        metric = -100*scores.mean()
        metric_std = (100*scores).std()
        metrics.append(metric)
        metric_stds.append(metric_std)
            
    best_metric_ind = np.argmin(metrics)
    current_metric = metrics[best_metric_ind]
    current_metrics.append(current_metric)

    current_metric_std = metric_stds[best_metric_ind]
    current_metric_stds.append(current_metric_std)
    
    opt_features.append(features[best_metric_ind])
    features = features.drop(features[best_metric_ind])

    print(current_metrics)
    print(current_metric_stds)
    print(opt_features)
    
    np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
    np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))
    np.savez_compressed('feature_selection/current_metric_stds.npz', a=np.array(current_metric_stds))

Features loop:   0%|          | 0/91 [00:00<?, ?it/s]

[3.8172847112571504]
[0.0035566912995007418]
['AccumCharge']


Features loop:   0%|          | 0/90 [00:00<?, ?it/s]

[3.8172847112571504, 1.6370731913953724]
[0.0035566912995007418, 0.002287511784975063]
['AccumCharge', 'R_cht']


Features loop:   0%|          | 0/89 [00:00<?, ?it/s]

[3.8172847112571504, 1.6370731913953724, 1.3473622855428409]
[0.0035566912995007418, 0.002287511784975063, 0.0021186522077396067]
['AccumCharge', 'R_cht', 'z_cc']


Features loop:   0%|          | 0/88 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))
np.savez_compressed('feature_selection/current_metric_stds.npz', a=np.array(current_metric_stds))

In [12]:
print(metrics, '\n')
print(metric_stds, '\n')
print(feature)

[1.245956701652736, 1.278174004720776, 1.2834816623682768, 1.3380338676295327, 1.3383747258618, 1.3073712999074987, 1.345516504067802, 1.345721629134331, 1.3060572265262744, 1.3468732988972114, 1.2763779869909124, 1.317818265214642, 1.3456416918107788, 1.3456344622966652, 1.3133105652775385, 1.3123316840974755, 1.3453624717819328, 1.3454538391557918, 1.3114130658321024, 1.346929641679613, 1.3208612118279834, 1.2807732374067504, 1.2875007328028285, 1.2578164607585869, 1.2622211863487798, 1.2243655986687676, 1.2440891449755052, 1.2785158001422803, 1.2936101037677608, 1.3161188793893919, 1.3255727414934286, 1.3350547739754224, 1.3402415424767258, 1.3350514925720025, 1.3221478207801813, 1.3119939352840133, 1.3063372848377104, 1.303514178480096, 1.3009993812599259, 1.2977030724940992, 1.2946580427140069, 1.2915387770891078, 1.2886208927827747, 1.2861882850827486, 1.283590440661464, 1.2793148989054448, 1.275112776249486, 1.272445739956457, 1.2789573754701045, 1.3005158733893356, 1.2843270974